### GoogleLeNet를 keras를 이용해서 구현

In [ ]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, AveragePoolig2D, Flatten, Concatnate
from keras.optimizers import SGD #확률적 경사 하강법 사용
from keras.callbacks import Callback

from keras.utils.import to_categorical #one-hot-encoding을 진행하기 위해서
from keras.datasets import cifar10
import numpy as np

In [ ]:
class LearningRateSchedule(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = None):
    if (epoch + 1) % 8 == 0:
      lr = K.get_value(self.model.optimizer.lr)
      K.set_value(self.model.optimizer.lr, lr*0.96)

#### LocalResponseNormalization
```py
import tensorflow as tf
tf.nn.local_response_normalization(
  input, depth_radius, bias, alpha, beta, name = None
)
```
- 일반적으로 요즘은 ReLU activation function을 사용한 경우에도 batch normalization을 사용하곤 한다.
- AlexNet에서 LRN을 사용한 이유는 ReLU가 양수의 방향으로는 입력의 값을 그대로 사용한다는 특성 떄문이다.
  - 그렇게 되면 Convolution / Pooling layer에서 매우 놏은 하나의 픽셀값이 주변의 픽셀에 영향을 주게 될 것이다.
  - 따라서 이런 부분을 방지하기 위해 **같은 위치에 있는 픽셀끼리의 정규화**를 진행하는 것이다.

In [ ]:
class LocalResponseNormalization(keras.layers.Layer):
  def __init__(self, n = 5, alpha = 1e-4, beta = 0.75, k = 2, **kwargs):
    self.n = n
    self.alpha = alpha
    self.beta = beta
    self.k = k
    super(LocalResponseNormalization, self).__init__(**kwargs)
  
  def build(self, input_shape):
    self.shape = input_shape
    super(LocalResopnseNormalization, self).build(input_shape)
  
  def call(self, x):
    _, r, c, f = self.shape
    squared = K.square(x)
    pooled = K.pool2d(squares, (self.n, self.n), strides = (1,1), padding = 'same', pool_mode = 'avg')
    summed = K.sum(pooled, axis = 3, keepdims = True)
    averaged = self.alpha * K.repeat_elements(summed, f, axis = 3)

    denom = K.pow(self.k + averaged, self.beta)

    return x/denom
  
  def get_output_shape(self, input_shape):
    return input_shape

In [ ]:
def Inception(input_tensor, filter_channels):
  filter_1x1, filter_3x3_R, filter_3x3, filter_5x5_R, filter_5x5, pool_proj = filter_channels

  branch_1 = Conv2D(filter_1x1, (1,1), strides = 1, padding = 'same',
                    activation = 'relu', kernel_initializer = 'he_normal')(input_tensor)

  branch_2 = Conv2D(filter_3x3_R, (1,1), strides = 1, padding = 'same',
                    activation = 'relu', kernel_initializer = 'he_normal')(input_tensor)
  branch_2 = Conv2D(filter_3x3, (3,3), strides = 1, padding = 'same', 
                    activation = 'relu', kernel_initializer = 'he_normal')(branch_2)

  branch_3 = Conv2D(filter_5x5_R, (1,1), strides = 1, padding = 'same', 
                    activation = 'relu', kernel_initializer = 'he_normal')(input_tensor)
  branch_3 = Conv2D(filter_5x5, (5,5), strides = 1, padding = 'same',
                    activation = 'relu', kernel_initializer = 'he_normal')(branch_3)

  branch_4 = MaxPooling2D((3,3), strides = 1, padding = 'same')(input_tensor)
  branch_4 = Conv2D((1,1), strides = 1, padding = 'same')(branch_4)

  Concat = Concatenate([branch_1, branch_2, branch_3, branch_4])

  return Concat

In [ ]:
def GoogLeNet(model_intput, classes = 10):
  # Main Classifier
  conv_1 = Conv2D(64, (7,7), strides = 2, padding = "same", activation = "relu")(model_input) #(112, 112, 64)
  pool_1 = MaxPooling2D((3, 3), strides = 2, padding = "same", activation = "relu")(conv_1) # (56, 56, 64)
  LRN_1 = LocalResponseNormalization()(pool_1) # (56, 56, 64)

  conv_2 = Conv2D(64, (1, 1), strides = 1, padding = "valid", activation = "relu")(LRN_1) # (56, 56, 64)
  conv_3 = Conv2D(192, (3, 3), strides = 1, padding = "same", activation = "relu")(conv_2) # (56, 56, 192)
  LRN_2 = LocalResponseNormalization()(conv_3) # (56, 56, 192)
  pool_2 = MaxPooling2D((3,3), strides = 2, padding = "same", activation = "relu")(LRN_2) # (28, 28, 192)

  inception_3a = inception(pool_2, [64, 96, 128, 16, 32, 32]) # (28, 28, 256) 
  # 256 = 
  inception_3b = inception(inception_3a, [128, 128, 192, 32, 96, 64]) # (28, 28, 480)

  pool_3 = MaxPooing2D((3, 3), strides = 2, padding = "same", activation = "relu")(inception_3b) # (14, 14, 480)

  inception_4a = inception(pool_3, [192, 96, 208, 16, 48, 64]) # (14, 14, 512)
  inception_4b = inception(inception_4a, [160, 112, 224, 24, 64, 64]) # (14, 14, 512)
  inception_4c = inception(inception_4b, [128, 128, 256, 24, 64, 64]) # (14, 14, 512)
  inception_4d = inception(inception_4c, [112, 144, 288, 32, 64, 64]) # (14, 14, 528)
  inception_4e = inception(inception_4d, [256, 160, 320, 32, 128, 128]) # (14, 14, 832)

  pool_4 = MaxPooling2D((3, 3), strides = 2, padding = "same")(inception_4e)

  inception_5a = inception(pool_4, [256, 160, 320, 32, 128, 128]) # (7, 7, 832)
  inception_5b = inception(inception_5a, [384, 192, 384, 48, 128, 128]) # (7, 7, 1024)

  avg_pool = GlobalAveragePooling2D()(inception_5b)
  dropout = Dropout(0.4)(avg_pool)

  linear = Dense(1000, activation = "relu")(dropout)

  model_output = Dense(classes, activation = "softmax", name = "main_classifier")(linear)

  # Auxiliary Classifier
  auxiliary_4a = AveragePooling2D((5, 5), strides = 3, padding = "valid")(inception_4a)
  auxiliary_4a = Conv2D(128, (1,1), strides = 1, padding = "same")(auxiliary_4a)
  auxiliary_4a = Flatten()(auxiliary_4a)
  auxiliary_4a = Dense(1024, activation = "relu")(auxiliary_4a)
  auxiliary_4a = Dropout(0.7)(auxiliary_4a)
  auxiliary_4a = Dense(classes, activation = "softmax", name = "auxiliary_4a")(auxiliary_4a)

  auxiliary_4d = AveragePooling2D((5, 5), strides = 3, padding = "valid")(inception_4d)
  auxiliary_4d = Conv2D(128, (1,1), strides = 1, padding = "same")(auxiliary_4d)
  auxiliary_4d = Flatten()(auxiliary_4d)
  auxiliary_4d = Dense(1024, activation = "relu")(auxiliary_4d)
  auxiliary_4d = Dropout(0.7)(auxiliary_4d)
  auxiliary_4d = Dense(classes, activation = "softmax", name = "auxiliary_4a")(auxiliary_4d)

  model = Model(model_input, [model_output, auxiliary_4a, auxiliary_4d])
  return model

In [ ]:
input_shape = (224, 224, 3)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = Upscale_Data(x_train, input_shape)/255.0
x_test = Upscale_Data(x_test, input_shape)/255.0

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

model_input = Input(shape = input_shape)

model = GoogLeNet(model_input, 10)

optimizer = SGD(momentum = 0.9)

model.compile(optimizer, loss = {'main_classifier' : 'categorical_crossentropy', 'auxiliary_4a' : 'categorical_crossentropy',
                                 'auxiliary_4d' : 'categorical_crossentropy'},
              loss_weights = {'main_classifier' : 1.0, 'auxiliary_4a' : 0.3, 'auciliary_4b' : 0.3},
              metrics = ['acc'])

model.fit(x_train,
          {'main_classifier' : y_train, 'auxiliary_4a' : y_train, 'auxiliary_4b' : y_train},
          epochs = 100, batch_size = 32, callbacks = LearningRateSchedule())


